In [2]:
import pandas as pd
import ast
import numpy as np
import re 
import sys
import os
import pathlib
import glob
import cv2 as cv
import importlib
from scipy.spatial.transform import Rotation as R

sys.path.append("../../")
from data_processor import readCSVWStrArray
from data_processor import IntrinsicCalib
from data_processor import timeStampAligner
from data_processor import videoToPng

In [277]:
importlib.reload(timeStampAligner)

<module 'data_processor.timeStampAligner' from '/home/fj/Projects/ARPA-H/Scripts/20241126_hand_eye_pivot_calibrations/../../data_processor/timeStampAligner.py'>

In [278]:
## Clean/Align the tracking data
basePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations")
timeStampAligner.processDirectory(basePath)

Found 4 NDI files and 1 video timestamp files.
Processing NDI file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_1.csv
Processing video timestamp file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/hand_eye_test_2.csv
(9381, 1, 4, 4)
Aligned data saved to /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_1_aligned.csv
Processing NDI file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_1_COPY.csv
Processing video timestamp file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/hand_eye_test_2.csv
(9381, 1, 4, 4)
Aligned data saved to /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_1_COPY_aligned.csv
Processing NDI file: /media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/ndi_hand_eye_test_2.csv
Processing video timestamp file: /media/fj/Data/Pro

In [279]:
videoBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations")
videoPath = videoBasePath / "hand_eye_test_2.avi"
imageBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations//Recordings/run_2/cam0")
imagePath = imageBasePath / "data"
timeStampsPath = videoBasePath / "hand_eye_test_2.csv"
timeStamps = np.genfromtxt(timeStampsPath, delimiter=",", dtype=np.double)[:, 1]

ndiDatapath = videoBasePath / "ndi_hand_eye_test_2_aligned.csv"



In [280]:
# Define the dimensions of the checkerboard (number of inner corners per row and column)
CHECKERBOARD_SHAPE = (10, 7)  # Adjust according to your checkerboard dimensions
CHECKERBOARD_GRID_SIZE = 5 # mm

In [30]:
videoToPng.saveFramesAsPng(videoPath, imagePath, timeStamps=timeStamps) # newSize=(640, 360), 


10868, 1732652848813

In [281]:
# Camera Calibration
images = glob.glob(str(imagePath / "*.png"))
print(f"# of Images: {len(images)}")
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 60, 0.0001)
ret, camera_matrix, dist_coeffs, rvecs, tvecs, fileNames = IntrinsicCalib.detectCheckerboardAndIntrinsicCalib(images, CHECKERBOARD_SHAPE=CHECKERBOARD_SHAPE, CHECKERBOARD_GRID_SIZE=CHECKERBOARD_GRID_SIZE, savePath=imageBasePath / "intrinsics", visualize=True, imageRange=[2500, len(images)-5000, 40], returnUsedFileNames=True)

# of Images: 10869
5880/10869
Error:
 0.6441704094449495
Camera Matrix:
 [[510.90831953   0.         691.04417775]
 [  0.         511.97818242 362.48606464]
 [  0.           0.           1.        ]]

Distortion Coefficients:
 [[-0.36928998  0.16981098  0.00220561 -0.0040024  -0.04389162]]


In [282]:
usedTimeStamps = [int(f.split("/")[-1][:-4]) for f in fileNames]
R_target2cam_vect = rvecs
t_target2cam = tvecs

ndiData = np.genfromtxt(ndiDatapath, delimiter=",") # [Translation, Quat]
ndiDataTimeStamps = ndiData[:, 0].astype(int)

usedNdiData = []
for usedTimeStamp in usedTimeStamps:
    idx = np.where(ndiDataTimeStamps == usedTimeStamp)[0][0]
    usedNdiData.append(ndiData[idx, :])

usedNdiData = np.array(usedNdiData)
t_gripper2base = [usedNdiData[i, 1:4] for i in range(usedNdiData.shape[0])]
R_gripper2base = [R.from_quat(usedNdiData[i, 4:]).as_matrix() for i in range(usedNdiData.shape[0])]

In [283]:
R_cam2gripper, t_cam2gripper = cv.calibrateHandEye(R_gripper2base, t_gripper2base, R_target2cam_vect, t_target2cam)

#### Hand-Eye Calibration Result
Using Method from [Tsai](https://ieeexplore.ieee.org/document/34770)
Other OpenCV implemented methods are listed [here](https://docs.opencv.org/4.5.4/d9/d0c/group__calib3d.html#gad10a5ef12ee3499a0774c7904a801b99).

Rotation and Translation
```
(array([[-0.05006572, -0.46495374,  0.88391823],
        [ 0.01838049, -0.88530718, -0.46464326],
        [ 0.99857678, -0.00701585,  0.05286962]]),
 array([[ 83.64166892],
        [ 77.06542646],
        [-21.23159487]]))
```





In [284]:
T_cam2gripper = np.concatenate((R_cam2gripper, t_cam2gripper), axis=-1)
print(T_cam2gripper)
T_cam2gripper = np.concatenate((T_cam2gripper, np.array([[0, 0, 0, 1]])), axis=0)

[[-6.75643936e-02 -4.92631609e-01  8.67611175e-01  2.65468573e+02]
 [-5.67763223e-03 -8.69394357e-01 -4.94086244e-01  9.98012267e-01]
 [ 9.97698761e-01 -3.83086146e-02  5.59431214e-02 -3.19069553e+00]]


In [285]:
np.save("./handEyeOutput.npy", T_cam2gripper)

## Error Evaluation

In [286]:
objp = np.zeros((CHECKERBOARD_SHAPE[0]*CHECKERBOARD_SHAPE[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD_SHAPE[0], 0:CHECKERBOARD_SHAPE[1]].T.reshape(-1, 2) * CHECKERBOARD_GRID_SIZE
objBases = []

for i, usedFileName in enumerate(fileNames):
    if i != 0:
        continue
    cur_t_gripper2base = t_gripper2base[i]
    cur_R_gripper2base = R_gripper2base[i]
    cur_T_gripper2base = np.eye(4)
    cur_T_gripper2base[:3, :3] = cur_R_gripper2base
    cur_T_gripper2base[:3, 3] = cur_t_gripper2base
    
    cur_T_cam2base = cur_T_gripper2base @ T_cam2gripper
    cur_T_base2cam = np.linalg.inv(cur_T_cam2base)
    cur_R_base2cam_vect = R.from_matrix(cur_T_base2cam[:3, :3]).as_rotvec()
    cur_t_base2cam = cur_T_base2cam[:3, 3]

    cur_R_target2cam_vect = R_target2cam_vect[0].squeeze()
    cur_t_target2cam = t_target2cam[0].squeeze()
    cur_T_target2cam = np.eye(4)
    cur_T_target2cam[:3, :3] = R.from_rotvec(cur_R_target2cam_vect).as_matrix()
    cur_T_target2cam[:3, 3] = cur_t_target2cam

    objpBase = np.ones([objp.shape[0], objp.shape[1]+1])
    objpBase[:, :3] = objp
    objpBase = cur_T_cam2base @ cur_T_target2cam @ objpBase.T
    # print(cur_T_target2cam)
    objpBase = objpBase[:3, :].T
    objBases.append(objpBase)
    
    img = cv.imread(usedFileName)
    cv.imshow('img', img)
    cv.waitKey(500)
objpBase = np.mean(objBases, axis=0)


In [287]:
cv.destroyAllWindows()

In [288]:
objp = np.zeros((CHECKERBOARD_SHAPE[0]*CHECKERBOARD_SHAPE[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD_SHAPE[0], 0:CHECKERBOARD_SHAPE[1]].T.reshape(-1, 2) * CHECKERBOARD_GRID_SIZE
avgRepErrors = [] # np.zeros(len(fileNames))

for i, usedFileName in enumerate(fileNames):
    cur_t_gripper2base = t_gripper2base[i]
    cur_R_gripper2base = R_gripper2base[i]
    cur_T_gripper2base = np.eye(4)
    cur_T_gripper2base[:3, :3] = cur_R_gripper2base
    cur_T_gripper2base[:3, 3] = cur_t_gripper2base
    
    cur_T_cam2base = cur_T_gripper2base @ T_cam2gripper
    cur_T_base2cam = np.linalg.inv(cur_T_cam2base)
    cur_R_base2cam_vect = R.from_matrix(cur_T_base2cam[:3, :3]).as_rotvec()
    cur_t_base2cam = cur_T_base2cam[:3, 3]

    # if i==0:
    #     cur_R_target2cam_vect = R_target2cam_vect[i].squeeze()
    #     cur_t_target2cam = t_target2cam[i].squeeze()
    #     cur_T_target2cam = np.eye(4)
    #     cur_T_target2cam[:3, :3] = R.from_rotvec(cur_R_target2cam_vect).as_matrix()
    #     cur_T_target2cam[:3, 3] = cur_t_target2cam

    #     objpBase = np.ones([objp.shape[0], objp.shape[1]+1])
    #     objpBase[:, :3] = objp
    #     objpBase = cur_T_cam2base @ cur_T_target2cam @ objpBase.T
    #     print(cur_T_target2cam)
    #     objpBase = objpBase[:3, :].T

    imgP, jac = cv.projectPoints(objpBase, cur_R_base2cam_vect, cur_t_base2cam, camera_matrix, dist_coeffs)
    img = cv.imread(usedFileName)
    # Draw and display the corners
    ret, groundTruthCorners = cv.findChessboardCorners(img, CHECKERBOARD_SHAPE, None)

    img = cv.drawChessboardCorners(img, CHECKERBOARD_SHAPE, imgP.astype(np.float32), True)
    if (ret):
        avgRepErrors.append(np.linalg.norm(imgP - groundTruthCorners, axis=-1))
        print(np.mean(avgRepErrors[-1]))
    # break
    cv.imshow('img', img)
    
    cv.waitKey(500)


1.0519396281262545
9.625186817352049
5.236842896586981
27.250302606562325
7.067117985253632
11.651827096723027
11.465124445482468
9.514819474673502
7.38153758588463
20.95926456053309
38.09531949516642
17.234248613685082
15.12858208024155
28.370944254222422
17.54494939683968
42.75713945113606
34.100691548126846
8.477792874235766
11.50634753399676
34.2078028476256
16.32486837644928
25.349884280363334
18.073683869925944
29.73578008849791


In [290]:
np.mean(avgRepErrors)

18.67133324198711

In [204]:
realavgRepErrors # = avgRepErrors

[array([[0.19726313],
        [1.80712401],
        [0.18436862],
        [0.99029046],
        [0.71913157],
        [1.28527217],
        [1.17180902],
        [1.07712916],
        [0.8835213 ],
        [0.67595966],
        [0.96887784],
        [0.82403755],
        [0.84591892],
        [1.17600205],
        [2.31770452],
        [0.45888035],
        [0.48393908],
        [0.17705758],
        [0.48453286],
        [0.15019106],
        [2.56251009],
        [1.9154503 ],
        [0.40273517],
        [0.68225639],
        [0.70349792],
        [2.37473085],
        [1.14043051],
        [1.25833035],
        [0.37803715],
        [0.58914069],
        [0.43932478],
        [0.33275692],
        [2.60349176],
        [1.79911693],
        [0.38892112],
        [2.65343433],
        [1.5034858 ],
        [1.88250254],
        [1.82251643],
        [0.2416438 ],
        [1.50649287],
        [0.40237354],
        [0.2282688 ],
        [0.70431689],
        [0.97372519],
        [1

In [207]:
np.mean(realavgRepErrors)

33.42113584035296

In [208]:
np.mean(avgRepErrors)

33.50371983146629

In [186]:
np.mean(avgRepErrors)

61.562108990361075

In [291]:
cv.destroyAllWindows()


In [ ]:
imgP, jac = cv.projectPoints(objp, rvecs[0], tvecs[0], camera_matrix, dist_coeffs)
print(objp)
print(imgP)

img = cv.imread(usedFileName)
ret, groundTruthCorners = cv.findChessboardCorners(img, CHECKERBOARD_SHAPE, None)
print(groundTruthCorners)
img = cv.drawChessboardCorners(img, CHECKERBOARD_SHAPE, imgP, True)
cv.imshow('img', img)
cv.waitKey(500)

[[ 0.  0.  0.]
 [ 5.  0.  0.]
 [10.  0.  0.]
 [15.  0.  0.]
 [20.  0.  0.]
 [25.  0.  0.]
 [30.  0.  0.]
 [35.  0.  0.]
 [40.  0.  0.]
 [45.  0.  0.]
 [ 0.  5.  0.]
 [ 5.  5.  0.]
 [10.  5.  0.]
 [15.  5.  0.]
 [20.  5.  0.]
 [25.  5.  0.]
 [30.  5.  0.]
 [35.  5.  0.]
 [40.  5.  0.]
 [45.  5.  0.]
 [ 0. 10.  0.]
 [ 5. 10.  0.]
 [10. 10.  0.]
 [15. 10.  0.]
 [20. 10.  0.]
 [25. 10.  0.]
 [30. 10.  0.]
 [35. 10.  0.]
 [40. 10.  0.]
 [45. 10.  0.]
 [ 0. 15.  0.]
 [ 5. 15.  0.]
 [10. 15.  0.]
 [15. 15.  0.]
 [20. 15.  0.]
 [25. 15.  0.]
 [30. 15.  0.]
 [35. 15.  0.]
 [40. 15.  0.]
 [45. 15.  0.]
 [ 0. 20.  0.]
 [ 5. 20.  0.]
 [10. 20.  0.]
 [15. 20.  0.]
 [20. 20.  0.]
 [25. 20.  0.]
 [30. 20.  0.]
 [35. 20.  0.]
 [40. 20.  0.]
 [45. 20.  0.]
 [ 0. 25.  0.]
 [ 5. 25.  0.]
 [10. 25.  0.]
 [15. 25.  0.]
 [20. 25.  0.]
 [25. 25.  0.]
 [30. 25.  0.]
 [35. 25.  0.]
 [40. 25.  0.]
 [45. 25.  0.]
 [ 0. 30.  0.]
 [ 5. 30.  0.]
 [10. 30.  0.]
 [15. 30.  0.]
 [20. 30.  0.]
 [25. 30.  0.]
 [30. 30. 

-1

In [125]:
cv.destroyAllWindows()


In [61]:
CHECKERBOARD_SHAPE

(10, 7)

In [79]:
objp

array([[ 0.,  0.,  0.],
       [ 5.,  0.,  0.],
       [10.,  0.,  0.],
       [15.,  0.,  0.],
       [20.,  0.,  0.],
       [25.,  0.,  0.],
       [30.,  0.,  0.],
       [35.,  0.,  0.],
       [40.,  0.,  0.],
       [45.,  0.,  0.],
       [ 0.,  5.,  0.],
       [ 5.,  5.,  0.],
       [10.,  5.,  0.],
       [15.,  5.,  0.],
       [20.,  5.,  0.],
       [25.,  5.,  0.],
       [30.,  5.,  0.],
       [35.,  5.,  0.],
       [40.,  5.,  0.],
       [45.,  5.,  0.],
       [ 0., 10.,  0.],
       [ 5., 10.,  0.],
       [10., 10.,  0.],
       [15., 10.,  0.],
       [20., 10.,  0.],
       [25., 10.,  0.],
       [30., 10.,  0.],
       [35., 10.,  0.],
       [40., 10.,  0.],
       [45., 10.,  0.],
       [ 0., 15.,  0.],
       [ 5., 15.,  0.],
       [10., 15.,  0.],
       [15., 15.,  0.],
       [20., 15.,  0.],
       [25., 15.,  0.],
       [30., 15.,  0.],
       [35., 15.,  0.],
       [40., 15.,  0.],
       [45., 15.,  0.],
       [ 0., 20.,  0.],
       [ 5., 20.

In [54]:
cur_t_gripper2base

array([   19.76,  -250.64, -1187.26])

In [ ]:
cur_t_base2cam

array([  122.72719248,  -240.27193646, -1238.99055081])

In [51]:
usedFileName

'/media/fj/Data/Projects/ARPA-H/data/20241126_hand_eye_pivot_calibrations/Recordings/run_2/cam0/data/1732652707900.png'

In [67]:
cv.destroyAllWindows()


In [37]:
cur_R_target2cam_vect.squeeze()

array([-0.36651051,  0.42104781,  0.08151665])

In [25]:
objp.shape

(70, 3)